In [5]:
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
import math

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

filepath = '/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/code/'

In [8]:
features = [
       'next_external_id', 
       'buzz_external_id', 
       'buzz_account_display_name', 
       'ext_table1_consumptions_by_type__link_clicks',
       'ext_table1_consumptions_by_type__other_clicks',
       'ext_table1_reactions_like_total',
       'ext_table1_stories_by_action_type__comment',
       'ext_table1_stories_by_action_type__like',
       'ext_table1_stories_by_action_type__share', 'ext_table1_engaged_fan',
       'ext_table1_fan_reach', 'ext_table1_impressions',
       'ext_table1_impressions_fan',
       'ext_twodays_consumptions_by_type__link_clicks',
       'ext_twodays_stories_by_action_type__share',
        ]

one_hour_features = [
                   'ext_table1_consumptions_by_type__link_clicks',
                   'ext_table1_consumptions_by_type__other_clicks',
                   'ext_table1_reactions_like_total',
                   'ext_table1_stories_by_action_type__comment',
                   'ext_table1_stories_by_action_type__like',
                   'ext_table1_stories_by_action_type__share', 
                   'ext_table1_engaged_fan',
                   'ext_table1_fan_reach',
                   'ext_table1_impressions',
                   'ext_table1_impressions_fan', 
                ]

two_day_features = [
    'ext_twodays_consumptions_by_type__link_clicks',
    'ext_twodays_stories_by_action_type__share',
]

cross_promotion_features = all_pages

In [ ]:
from sklearn import linear_model
clf = linear_model.LinearRegression()

print("LINEAR REGRESSION!")

train_df = pd.read_csv(filepath+'previous-to-next-page-info.csv')
all_pages = list(train_df.buzz_account_display_name.unique())

print("WITH cross-promotion features")

page_infos = ['buzz_account_display_name', 'next_page']

# train_features = one_hour_features
# train_features = one_hour_features + page_infos
train_features = one_hour_features + page_infos + cross_promotion_features
# train_features = cross_promotion_features + one_hour_features
# train_features = two_day_features + one_hour_features + cross_promotion_features

for feats in (one_hour_features):
    train_df[feats] = np.log(train_df[feats] + 1)

# lm = linear_model.LinearRegression(fit_intercept=True, normalize=True)
cv = 10

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1).astype(int)
train_df['share_bucket'] = np.log(train_df['next_shares']+1).astype(int)

print("No. of data-points = ", len(train_df))

# y_clicks = train_df['next_clicks']
# y_share = train_df['next_shares']
y_clicks = train_df['clicks_bucket']
y_share = train_df['share_bucket']

###generate DFs for analysis - X and Y
X = train_df[train_features]
####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['buzz_account_display_name', 'next_page'])

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
    print("Fold = ", fold)
#     print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
        
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
#     print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

LINEAR REGRESSION!
WITH cross-promotion features
No. of data-points =  18438
Fold =  0
Fold =  1
Fold =  2
Fold =  3
Fold =  4


In [13]:
# from sklearn import linear_model
# clf = linear_model.LinearRegression()
# print("LINEAR REGRESSION!")

from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(max_depth = 10)
print("TREE REGRESSION!")

train_df = pd.read_csv(filepath+'previous-to-next-page-info.csv')
all_pages = list(train_df.buzz_account_display_name.unique())

print("WITH cross-promotion features")

page_infos = ['buzz_account_display_name', 'next_page']

# train_features = one_hour_features
train_features = one_hour_features + page_infos
# train_features = one_hour_features + two_day_features + page_infos
# train_features = cross_promotion_features + one_hour_features

# train_features = two_day_features + one_hour_features + cross_promotion_features

# for feats in (one_hour_features):
#     train_df[feats] = np.log(train_df[feats] + 1)

# lm = linear_model.LinearRegression(fit_intercept=True, normalize=True)
cv = 10

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1).astype(int)
train_df['share_bucket'] = np.log(train_df['next_shares']+1).astype(int)

print("No. of data-points = ", len(train_df))

y_clicks = train_df['next_clicks']
y_share = train_df['next_shares']
# y_clicks = train_df['clicks_bucket']
# y_share = train_df['share_bucket']

###generate DFs for analysis - X and Y
X = train_df[train_features]
####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['buzz_account_display_name', 'next_page'])

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
    print("Fold = ", fold)
#     print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
        
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
#     print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

TREE REGRESSION!
WITH cross-promotion features
No. of data-points =  18438
Fold =  0
Fold =  1
Fold =  2
Fold =  3
Fold =  4
Fold =  5
Fold =  6
Fold =  7
Fold =  8
Fold =  9
Shares performance: Accuracy =  0.848103180387  ; Error (/100) =  0.51315697138
Clicks performance: Accuracy =  0.840930107466  ; Error (/100) =  0.420025453165


In [14]:
print("ONLY 1 HOUR PERFORMANCE IN PREVIOUS PAGE")
print('SHARES:')
res = ols(y = y_share, x = X)
print(res)
print('CLICKS:')
res = ols(y =y_clicks, x = X)
print(res)

ONLY 1 HOUR PERFORMANCE IN PREVIOUS PAGE
SHARES:


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/pandas/stats/ols.py:336: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self._var_beta_raw))
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/eshwarchandrasekharan/anaconda3/


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <ext_table1_consumptions_by_type__link_clicks>
             + <ext_table1_consumptions_by_type__other_clicks> + <ext_table1_reactions_like_total>
             + <ext_table1_stories_by_action_type__comment>
             + <ext_table1_stories_by_action_type__like> + <ext_table1_stories_by_action_type__share>
             + <ext_table1_engaged_fan> + <ext_table1_fan_reach>
             + <ext_table1_impressions> + <ext_table1_impressions_fan>
             + <buzz_account_display_name_Adam Ellis> + <buzz_account_display_name_Another Round>
             + <buzz_account_display_name_Bien Tasty>
             + <buzz_account_display_name_Buy Me That> + <buzz_account_display_name_BuzzFeed>
             + <buzz_account_display_name_BuzzFeed Animals>
             + <buzz_account_display_name_BuzzFeed Australia> + <buzz_account_display_name_BuzzFeed BFF>
             + <buzz_account_display_name_BuzzFee